In [ ]:
## Dependencies

import pandas as pd
import numpy as np

import sqlite3

# db_path = '../data/2022-2023 Season Data.db' # Set FOr 2022-2023 Season

# db_path = '../TEMP/2023_Season_Nov 2_Game_Stats.db' # Set For 2023-2024 Season

# db_path = '../data/db/Current_Season_YTD_Game_Stats.db' # Set For 2023-2024 Season

# db_path = '../TEMP/2022_Game_Stats.db' # 2022 Season

# db_path = '../data/db/2021_Season_v1_Game_Stats.db' # 2021 Season

# db_path = '../TEMP/Current_Season_YTD_Game_Stats.db'

db_path = '../../data/db/2022_Game_Stats_Cleaned.db'



conn = sqlite3.connect(db_path)

## Get the Year to Date Player Totals in dataframe
# df = pd.read_sql_query("SELECT * FROM 'player_stats_ytd'", conn)

# List tables
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

## Get the Year to Date Player Totals in dataframe
df = pd.read_sql_query("SELECT * FROM 'player_stats_ytd'", conn)

## Total Goals by Team & Number of Scorers by Team

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

player_stats = df.copy()
# Group by team and aggregate necessary statistics
team_stats = player_stats.groupby('Team').agg(
    Total_Goals=pd.NamedAgg(column='G', aggfunc='sum'),
    Different_Scorers=pd.NamedAgg(column='G', aggfunc=lambda x: (x>0).sum()),
    Top_Scorer_Goals=pd.NamedAgg(column='G', aggfunc='max')
).reset_index()

# Sort the teams by total goals
team_stats = team_stats.sort_values(by='Total_Goals', ascending=False)

# Plotting
plt.figure(figsize=(15, 10))

# Plot for total goals by team
plt.subplot(2, 1, 1)
sns.barplot(x='Total_Goals', y='Team', data=team_stats, palette='coolwarm')
plt.title('Total Goals by Team')
plt.xlabel('Total Goals')
plt.ylabel('Team')

# Plot for number of different scorers by team
plt.subplot(2, 1, 2)
sns.barplot(x='Different_Scorers', y='Team', data=team_stats, palette='coolwarm')
plt.title('Number of Different Scorers by Team')
plt.xlabel('Number of Different Scorers')
plt.ylabel('Team')

plt.tight_layout()
plt.show()


In [ ]:
# Calculate the percentage of goals scored by the top scorer for each team
team_stats['Top_Scorer_Percentage'] = (team_stats['Top_Scorer_Goals'] / team_stats['Total_Goals']) * 100

# Sorting by top scorer percentage
team_stats_sorted_by_top_scorer = team_stats.sort_values(by='Top_Scorer_Percentage', ascending=False)

# Plotting
plt.figure(figsize=(15, 6))
sns.barplot(x='Top_Scorer_Percentage', y='Team', data=team_stats_sorted_by_top_scorer, palette='viridis')
plt.title('Percentage of Team Goals Scored by Top Scorer')
plt.xlabel('Percentage of Goals by Top Scorer')
plt.ylabel('Team')
plt.show()


## Scatter plots

In [ ]:
# Prepare data for scatter plots
goal_scorers_per_team = player_stats[player_stats['G'] > 0].groupby('Team')['G'].agg(['count', 'sum']).reset_index()
assist_makers_per_team = player_stats[player_stats['A'] > 0].groupby('Team')['A'].agg(['count', 'sum']).reset_index()

# Plotting
fig, ax = plt.subplots(1, 2, figsize=(18, 8))

# Goals Scatter Plot
ax[0].scatter(goal_scorers_per_team['count'], goal_scorers_per_team['sum'], alpha=0.5)
ax[0].scatter(goal_scorers_per_team[goal_scorers_per_team['Team'] == 'Michigan State']['count'],
              goal_scorers_per_team[goal_scorers_per_team['Team'] == 'Michigan State']['sum'],
              color='red', label='Michigan State')
ax[0].scatter(goal_scorers_per_team[goal_scorers_per_team['Team'] == 'Ferris State']['count'],
              goal_scorers_per_team[goal_scorers_per_team['Team'] == 'Ferris State']['sum'],
              color='green', label='Ferris State')
ax[0].set_title('Goal Scorers per Team')
ax[0].set_xlabel('Number of Different Goal Scorers')
ax[0].set_ylabel('Total Goals Scored')
ax[0].legend()

# Assists Scatter Plot
ax[1].scatter(assist_makers_per_team['count'], assist_makers_per_team['sum'], alpha=0.5)
ax[1].scatter(assist_makers_per_team[assist_makers_per_team['Team'] == 'Michigan State']['count'],
              assist_makers_per_team[assist_makers_per_team['Team'] == 'Michigan State']['sum'],
              color='red', label='Michigan State')
ax[1].scatter(assist_makers_per_team[assist_makers_per_team['Team'] == 'Ferris State']['count'],
              assist_makers_per_team[assist_makers_per_team['Team'] == 'Ferris State']['sum'],
              color='green', label='Ferris State')
ax[1].set_title('Assist Makers per Team')
ax[1].set_xlabel('Number of Different Assist Makers')
ax[1].set_ylabel('Total Assists Made')
ax[1].legend()

plt.tight_layout()
plt.show()
